In [16]:
import pandas as pd
import numpy as np
import duckdb

In [17]:
import os
import openai

In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [19]:
import json
import re
from tqdm import tqdm

In [20]:
import time

# Clean page detail data

In [21]:
df_page = pd.read_excel('dataset_facebook-pages-scraper_2025-04-27_17-56-14-278.xlsx')

In [22]:
df_page_cleaned = duckdb.query("""
select title, likes as page_like
,  facebookURL as page_url
, case
when facebookURL like '%KTAMSmartTrade%' then 'KTAM'
when facebookURL like '%scbam%' then 'SCBAM'
when facebookURL like '%krungsri%' then 'KSAM'
when facebookURL like '%Eastspring%' then 'EASTSPRING'
when facebookURL like '%kasikornasset%' then 'KASSET'
when facebookURL like '%mfc%' then 'MFC'
when facebookURL like '%bblam%' then 'BBLAM'
end fund_house
from df_page where title is not null
""").to_df()

In [23]:
df_page_cleaned

,title,page_like,page_url,fund_house
0,KTAM Smart Trade | Bangkok,500342,https://www.facebook.com/KTAMSmartTrade,KTAM
1,SCBAM บริษัทหลักทรัพย์จัดการกองทุน ไทยพาณิชย์ ...,137537,https://www.facebook.com/scbam.official,SCBAM
2,Krungsri Asset Management | Bangkok,80095,https://www.facebook.com/krungsriasset.official,KSAM
3,Eastspring Thailand,107050,https://www.facebook.com/Eastspring.Thailand,EASTSPRING
4,KAsset | Bangkok,295983,https://www.facebook.com/kasikornasset,KASSET
5,MFC Asset Management | Bangkok,23678,https://www.facebook.com/mfcfunds,MFC
6,BBLAM | Bangkok,164170,https://www.facebook.com/bblam.Fanpage,BBLAM


# Clean post data

In [24]:
df_post = pd.read_excel('dataset_facebook-posts-scraper_2025-04-30_08-25-49-186.xlsx')

In [25]:
df_post['text_ocr_extracted'] = df_post['media/0/ocrText'].str.extract(r"'([^']*)'")

In [26]:
df_post_cleaned = duckdb.query("""
select 
concat('A',postid) as post_id
, cast(time as datetime) as datetime
, strftime(cast(time as datetime), '%Y-%W') AS year_week

, case 
when "media/0/__typename" = 'Video' then 'Video'
when "media/0/__typename" = 'Photo' then 'Photo'
when "media/0/__typename" = 'GenericAttachmentMedia' then 'Multiple media'
else 'Text'
end as media_type

, case when likes is null then 0.0 else likes end as no_interaction
, case when comments is null then 0.0 else comments end as no_comment
, case when shares is null then 0.0 else shares end as no_share
                          
, case when text is null then text_ocr_extracted else text end as title
, left(case when text is null then text_ocr_extracted else text end, 50) as title_50

, url as post_url

from df_post
where "media/0/__typename" is null or "media/0/__typename" != 'ProfilePicAttachmentMedia'
""").to_df()

In [27]:
df_post_cleaned

,post_id,datetime,year_week,media_type,no_interaction,no_comment,no_share,title,title_50,post_url
0,A1029408782560119,2025-01-02 03:00:07,2025-00,Text,8.0,3.0,0.0,⚠️ รู้หรือไม่? ⚠️ \n80% ของบริษัทใน Fortune Gl...,⚠️ รู้หรือไม่? ⚠️ \n80% ของบริษัทใน Fortune Gl...,https://www.facebook.com/kasikornasset/posts/p...
1,A1030140475820283,2025-01-03 03:00:05,2025-00,Text,42.0,2.0,8.0,"New Year, New Me !!\nมนุษย์เงินเดือน อยากย้ายง...","New Year, New Me !!\nมนุษย์เงินเดือน อยากย้ายง...",https://www.facebook.com/kasikornasset/posts/p...
2,A1005583014933067,2025-01-03 04:00:10,2025-00,Text,8.0,2.0,2.0,บลจ.ไทยพาณิชย์ เปิดเสนอขายกองทุน Term Fund อาย...,บลจ.ไทยพาณิชย์ เปิดเสนอขายกองทุน Term Fund อาย...,https://www.facebook.com/scbam.official/posts/...
3,A1030341595800171,2025-01-03 10:00:08,2025-00,Text,45.0,6.0,5.0,⚠️ หุ้นไทย วันแรก -21 จุด ⚠️ \nSET อาจดูไม่ค่อ...,⚠️ หุ้นไทย วันแรก -21 จุด ⚠️ \nSET อาจดูไม่ค่อ...,https://www.facebook.com/kasikornasset/posts/p...
4,A1030380105796320,2025-01-03 11:00:06,2025-00,Text,15.0,0.0,1.0,เริ่มต้นปีใหม่ เพิ่มความมั่งคั่งให้พอร์ตลงทุนต...,เริ่มต้นปีใหม่ เพิ่มความมั่งคั่งให้พอร์ตลงทุนต...,https://www.facebook.com/kasikornasset/posts/p...
...,...,...,...,...,...,...,...,...,...,...
1095,A1123948063106190,2025-04-30 03:32:06,2025-17,Photo,12.0,0.0,4.0,ห้ามพลาดกับ 2 กองทุน IPO ใหม่❗️Thai ESGX จากกส...,ห้ามพลาดกับ 2 กองทุน IPO ใหม่❗️Thai ESGX จากกส...,https://www.facebook.com/kasikornasset/posts/p...
1096,A1102537441907628,2025-04-30 04:08:17,2025-17,Photo,3.0,0.0,0.0,สรุปภาวะตลาดประจำวัน | 30 เมษายน 2568\n#ตลาดหุ...,สรุปภาวะตลาดประจำวัน | 30 เมษายน 2568\n#ตลาดหุ...,https://www.facebook.com/mfcfunds/posts/pfbid0...
1097,A695261349559696,2025-04-30 04:53:12,2025-17,Video,1.0,0.0,0.0,ดอกเบี้ยขาลง ตลาดหุ้นก็ลงเยอะ กองทุนไหนมีโอกาส...,ดอกเบี้ยขาลง ตลาดหุ้นก็ลงเยอะ กองทุนไหนมีโอกาส...,https://www.facebook.com/reel/1054382436567956/
1098,A1230360845766201,2025-04-30 04:59:43,2025-17,Photo,4.0,0.0,2.0,PRIME Pick: สัปดาห์นี้ [ 28 เม.ย. - 2 พ.ค. 68 ...,PRIME Pick: สัปดาห์นี้ [ 28 เม.ย. - 2 พ.ค. 68 ...,https://www.facebook.com/Eastspring.Thailand/p...


In [28]:
df_post_cleaned = duckdb.query("""
select *
, case
when post_url like '%KTAMSmartTrade%' then 'KTAM'
when post_url like '%scbam%' then 'SCBAM'
when post_url like '%krungsri%' then 'KSAM'
when post_url like '%Eastspring%' then 'EASTSPRING'
when post_url like '%kasikornasset%' then 'KASSET'
when post_url like '%mfc%' then 'MFC'
when post_url like '%bblam%' then 'BBLAM'
end fund_house
from df_post_cleaned
""").to_df()

In [29]:
df_post_cleaned.head()

,post_id,datetime,year_week,media_type,no_interaction,no_comment,no_share,title,title_50,post_url,fund_house
0,A1029408782560119,2025-01-02 03:00:07,2025-00,Text,8.0,3.0,0.0,⚠️ รู้หรือไม่? ⚠️ \n80% ของบริษัทใน Fortune Gl...,⚠️ รู้หรือไม่? ⚠️ \n80% ของบริษัทใน Fortune Gl...,https://www.facebook.com/kasikornasset/posts/p...,KASSET
1,A1030140475820283,2025-01-03 03:00:05,2025-00,Text,42.0,2.0,8.0,"New Year, New Me !!\nมนุษย์เงินเดือน อยากย้ายง...","New Year, New Me !!\nมนุษย์เงินเดือน อยากย้ายง...",https://www.facebook.com/kasikornasset/posts/p...,KASSET
2,A1005583014933067,2025-01-03 04:00:10,2025-00,Text,8.0,2.0,2.0,บลจ.ไทยพาณิชย์ เปิดเสนอขายกองทุน Term Fund อาย...,บลจ.ไทยพาณิชย์ เปิดเสนอขายกองทุน Term Fund อาย...,https://www.facebook.com/scbam.official/posts/...,SCBAM
3,A1030341595800171,2025-01-03 10:00:08,2025-00,Text,45.0,6.0,5.0,⚠️ หุ้นไทย วันแรก -21 จุด ⚠️ \nSET อาจดูไม่ค่อ...,⚠️ หุ้นไทย วันแรก -21 จุด ⚠️ \nSET อาจดูไม่ค่อ...,https://www.facebook.com/kasikornasset/posts/p...,KASSET
4,A1030380105796320,2025-01-03 11:00:06,2025-00,Text,15.0,0.0,1.0,เริ่มต้นปีใหม่ เพิ่มความมั่งคั่งให้พอร์ตลงทุนต...,เริ่มต้นปีใหม่ เพิ่มความมั่งคั่งให้พอร์ตลงทุนต...,https://www.facebook.com/kasikornasset/posts/p...,KASSET


# Set up API

In [ ]:
OPENAI_API_KEY = ''
llm_model = 'gpt-4.1-2025-04-14'
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=llm_model, temperature=0)

# Tagging AI

In [31]:
template_tagging = """
    You are tasked with tagging a post based on the given schema.
    Please follow 'TaggingSchema'
    **Post**: {post_text}
"""

In [32]:
prompt_tagging = ChatPromptTemplate.from_template(template_tagging)

In [33]:
TaggingSchema = {
    "title": "TaggingSchema",
    "type": "object",
    "description": "Schema for tagging post category, extracting fund codes, cleaning text, and extracting important keywords.",
    "properties": {
        "post_category": {
            "type": "string",
            "description": """
            Classify the post into one of the predefined categories.
            Choose only one category from the list below and do not create a new category:
            - News
            - Product Recommendation
            - Special Day
            - Application
            - Announcement
            If no suitable category is found, default to 'Uncategorized'.
            """,
            "enum": ["News", "Product Recommendation", "Special Day", "Application"],
            "default": "Uncategorized"
        },
        "fund_code": {
            "type": "array",
            "description": """
            Extract an array of fund codes mentioned in the post.
            Each fund code must be a string.
            Important:
            - Only include valid fund codes (e.g., mutual fund IDs).
            - Do not include fund types (e.g., 'Equity Fund') in this field.
            """,
            "items": {
                "type": "string"
            },
            "default": []
        },
        "important_keywords": {
            "type": "array",
            "description": """
            Extract an array of important keywords from the post.
            Important:
            - Not include valid fund codes (e.g., mutual fund IDs).
            - Keywords should be nouns or noun phrases relevant to the post content (e.g., 'promotion', 'new fund', 'NAV update').
            - Focus on product names, event names, or other critical terms.
            """,
            "items": {
                "type": "string"
            },
            "default": []
        }
    },
    "required": ["post_category", "fund_code", "important_keywords"]
}

In [34]:
llm_structured_tagging = llm.with_structured_output(TaggingSchema)

In [35]:
chain_tagging = prompt_tagging | llm_structured_tagging

In [36]:
def tagging_extraction(user_input, post_id):
    try:
        response = chain_tagging.invoke(user_input)

        if not isinstance(response, dict):
            return None

        response["post_id"] = post_id
        return response

    except Exception:
        return None

In [37]:
test_input = """กองทุนใหม่ SCBFLOAT1YC โอกาสเติบโตกับกลยุทธ์ลงทุนลดเสี่ยงขาดทุนเงินต้น จากตราสารหนี้คุณภาพ พร้อมสร้างผลตอบแทนแบบลอยตัวตามการปรับตัวของ THOR

เสนอขายครั้งแรก: 10 เม.ย. 68 - 22 เม.ย. 68 
เสนอขายผ่านช่องทาง SCB, SCBAM และ InnovestX เท่านั้น 
ห้ามขายผู้ลงทุนรายย่อย ระยะเวลาลงทุน 1 ปี ลงทุนขั้นต่ำ 500,000 บาท

กลยุทธ์การลงทุนที่พร้อมดูแลเงินต้น 
ส่วนที่ 1: ลงทุนตราสารหนี้ และเงินฝากคุณภาพดี ระดับ Investment Grade รับเงินลงทุนคืนพร้อมดอกเบี้ยเมื่อครบกำหนด

สร้างโอกาสรับผลตอบแทนเพิ่มขึ้น 
ส่วนที่ 2: ทำธุรกรรม Interest Rate Swap อ้างอิงกับอัตราดอกเบี้ย THOR โดยนักลงทุนจะได้รับผลตอบแทนในลักษณะลอยตัวตามการปรับตัวของ THOR (จ่ายผลตอบแทนเมื่อครบอายุโครงการ)

ข้อมูลกองทุนเพิ่มเติม : https://scbam.info/4juYIt9
SCBAM Client Relations 0 2777 7777

Complex Fund ผู้ลงทุนไม่สามารถขายคืนหน่วยลงทุนในช่วงเวลา 1 ปีได้ ดังนั้น หากมีปัจจัยลบที่ส่งผลกระทบต่อการลงทุนดังกล่าวผู้ลงทุนอาจสูญเสียเงินลงทุนจำนวนมาก
• กองทุนเปิดไทยพาณิชย์ Floating Rate Complex Return 1YC ห้ามขายผู้ลงทุนรายย่อย (SCBFLOAT1YC) ความเสี่ยง 4 เสี่ยงปานกลางค่อนข้างต่ำ • การลงทุนในผลิตภัณฑ์ในตลาดทุนที่มีความเสี่ยงสูงหรือมีความซับซ้อนซึ่งมีปัจจัยอ้างอิง มีความแตกต่างจากการลงทุนในปัจจัยอ้างอิงโดยตรง จึงอาจทำให้ราคาของผลิตภัณฑ์ในตลาดทุนดังกล่าว มีความผันผวนแตกต่างจากราคาของปัจจัยอ้างอิงได้ • กองทุนยังคงมีความเสี่ยงผิดชำระหนี้ (default risk) ที่เกิดขึ้นจากการผิดชำระหนี้ของผู้ออกตราสาร/เงินฝาก ซึ่งอาจส่งผลให้ผู้ลงทุน ไม่ได้รับเงินต้นคืนเต็มจำนวนได้ /ราคาของสัญญา ขึ้นอยู่กับการตกลงกันระหว่างคู่สัญญา กองทุนยังคงมีความเสี่ยงผิดนัดชำระหนี้ (default risk) ที่เกิดขึ้นจากการผิดนัดชำระหนี้ของผู้ออกสัญญา/ คู่สัญญา ซึ่งอาจส่งผลให้ผู้ลงทุน ไม่ได้รับผลตอบแทนจากสัญญาได้ บริษัทจัดการขอสงวนสิทธิอาจเปลี่ยนแปลงผลตอบแทนดังกล่าวได้หากสภาวะตลาดมีการเปลี่ยนแปลง โดยไม่ต่ำกว่าอัตรา ที่ระบุไว้ในโครงการ (บริษัทจัดการจะระบุอัตราที่แน่นอนในหนังสือชี้ชวนส่วนสรุปข้อมูลสำคัญ Factsheet ในช่วงเสนอขายครั้งแรก (IPO)) • กองทุนมีความเสี่ยงสูง หรือซับซ้อน ผู้ลงทุนควร ทำความเข้าใจลักษณะสินค้า เงื่อนไขผลตอบแทน และความเสี่ยงที่เกี่ยวข้อง รวมถึงควรขอคำแนะนำเพิ่มเติมจากผู้ประกอบธุรกิจก่อนตัดสินใจลงทุน • สอบถามรายละเอียด เพิ่มเติม และขอรับหนังสือชี้ชวนได้ที่ธนาคารไทยพาณิชย์ หรือบลจ.ไทยพาณิชย์ โทร. 02-777-7777 เว็บไซต์ scbam.com  #SCBAM #YourTrustedPartner
"""
test_id = 'dfweg23532f'

In [38]:
tagging_extraction(user_input=test_input, post_id=test_id)

{'post_category': 'Product Recommendation',
 'fund_code': ['SCBFLOAT1YC'],
 'important_keywords': ['กองทุนใหม่',
  'ตราสารหนี้คุณภาพ',
  'ผลตอบแทนลอยตัว',
  'THOR',
  'Interest Rate Swap',
  'SCBAM',
  'Floating Rate Complex Return 1YC',
  'เสนอขายครั้งแรก',
  'ลงทุนขั้นต่ำ',
  'ความเสี่ยงปานกลางค่อนข้างต่ำ'],
 'post_id': 'dfweg23532f'}

# Get tagging

In [39]:
tagging_results = []

In [40]:
for idx, row in tqdm(df_post_cleaned.iterrows(), total=len(df_post_cleaned), desc="Analyzing Mutual Fund Posts"):
    title = row['title']
    post_id = row['post_id']
    
    result = tagging_extraction(user_input=title, post_id=post_id)
    if result:tagging_results.append(result)

Analyzing Mutual Fund Posts: 100%|██████████| 1100/1100 [30:21<00:00,  1.66s/it] 


In [57]:
df_tagged_full = pd.DataFrame(tagging_results)
df_tagged_1 = df_tagged_full
df_tagged_2 = df_tagged_full[['post_id', 'fund_code']].explode('fund_code').dropna()

# Fund type AI

In [42]:
template_fundtype = """
    You are tasked with tagging a post based on the given schema.
    Please follow 'FundtypeSchema'
    **Post**: {post_text}
"""

In [43]:
prompt_fundtype = ChatPromptTemplate.from_template(template_fundtype)

In [44]:
FundtypeSchema = {
    "title": "FundtypeSchema",
    "type": "object",
    "description": "Schema for extracting a single fund type from fund codes mentioned in the post. Fund types are classified based on naming patterns and known identifiers.",
    "properties": {
        "fund_type": {
            "type": "string",
            "description": """
            Extract one fund type based on the fund code mentioned in the post.

            Choose only one from the list below:
            - Money Market Fund
            - Fixed Income Fund
            - Term Fund
            - Balance Fund
            - Thai Equity Fund
            - Index Fund
            - Foreign Equity Fund
            - Alternative Fund
            - SSF Fund
            - RMF Fund
            - LTF Fund
            - Thai ESG Fund

            Term Fund is hardest one if fund code contain duration such as 1Y 6M 24M that is Term Fund for sure.
            If multiple fund types are mentioned, return the most specific one based on naming conventions.
            If no match is found, return "Uncategorized".
            """,
            "enum": [
                "Money Market Fund",
                "Fixed Income Fund",
                "Term Fund",
                "Balance Fund",
                "Thai Equity Fund",
                "Foreign Equity Fund",
                "Alternative Fund",
                "SSF Fund",
                "RMF Fund",
                "LTF Fund",
                "Thai ESG Fund",
                "Uncategorized"
            ],
            "default": "Uncategorized"
        }
    },
    "required": ["fund_type"]
}

In [45]:
llm_structured_fundtype = llm.with_structured_output(FundtypeSchema)

In [46]:
chain_fundtype = prompt_fundtype | llm_structured_fundtype

In [47]:
def fundtype_extraction(user_input, post_id):
    try:
        response = chain_fundtype.invoke(user_input)

        if not isinstance(response, dict):
            return None

        response["post_id"] = post_id
        response["fund_code"] = user_input
        return response

    except Exception:
        return None

In [48]:
test_input = 'SCBFLOAT1YC'
post_id = 'afewf'

In [49]:
fundtype_extraction(user_input=test_input, post_id=post_id)

{'fund_type': 'Term Fund', 'post_id': 'afewf', 'fund_code': 'SCBFLOAT1YC'}

# Get fundtype

In [50]:
fundtype_results = []

In [51]:
df_tagged_2

,post_id,fund_code
8,A1008518471306188,SCBS&P500
8,A1008518471306188,SCBNDQA
11,A1009271594564209,SCBLT1FUND
11,A1009271594564209,SCBLT2FUND
11,A1009271594564209,SCBLT3FUND
...,...,...
1097,A695261349559696,KFINFRA
1098,A1230360845766201,ES-GINFRA-A
1098,A1230360845766201,ES-NDQPIN-UH
1098,A1230360845766201,ES-INDAE


In [52]:
for idx, row in tqdm(df_tagged_2.iterrows(), total=len(df_tagged_2), desc="Analyzing Mutual Fund Posts"):
    title = row['fund_code']
    post_id = row['post_id']
    
    result = fundtype_extraction(user_input=title, post_id=post_id)
    if result:fundtype_results.append(result)

Analyzing Mutual Fund Posts: 100%|██████████| 878/878 [11:32<00:00,  1.27it/s]


In [53]:
df_tagged_3 = pd.DataFrame(fundtype_results)

In [54]:
df_tagged_3

,fund_type,post_id,fund_code
0,Index Fund,A1008518471306188,SCBS&P500
1,Fixed Income Fund,A1008518471306188,SCBNDQA
2,LTF Fund,A1009271594564209,SCBLT1FUND
3,LTF Fund,A1009271594564209,SCBLT2FUND
4,LTF Fund,A1009271594564209,SCBLT3FUND
...,...,...,...
873,Alternative Fund,A695261349559696,KFINFRA
874,Alternative Fund,A1230360845766201,ES-GINFRA-A
875,Foreign Equity Fund,A1230360845766201,ES-NDQPIN-UH
876,Foreign Equity Fund,A1230360845766201,ES-INDAE


# Export

In [ ]:
df_final_1 = duckdb.query("""
select a.*
, b.post_category, b.important_keywords, b.fund_code as fund_code_list
, c.page_like
from df_post_cleaned a
inner join df_tagged_1 b
on a.post_id = b.post_id
inner join df_page_cleaned c
on a.fund_house = c.fund_house
where b.post_category is not null
""").to_df()

In [62]:
df_final_1.to_excel('tagging.xlsx', index=False)

In [63]:
df_final_2 = duckdb.query("""
select a.*
from df_tagged_3 a
inner join (select distinct post_id from df_final_1) b
on a.post_id = b.post_id
""").to_df()

In [64]:
df_final_2.to_excel('fund_type.xlsx', index=False)